<a href="https://colab.research.google.com/github/Equisoain/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_de_Optimizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Maddi Equisoain Redin  <br>
Url: https://github.com/Equisoain/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Maddi_Equisoain.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1r4Hc1GOdsVvvYZdzV9shgDHJjJRHo1eN <br>
Problema:

>**2. Organizar los horarios de partidos de una jornada de La Liga**

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes:


<table>
    <tr>
        <td>Lunes</td>
        <td>20</td>
    </tr>
    <tr>
        <td>Sábado</td>
        <td>12, 16, 18, 20</td>
    </tr>
    <tr>
        <td>Domingo</td>
        <td>12, 16, 18, 20</td>
    </tr>
    <tr>
        <td>Lunes</td>
        <td>20</td>
    </tr>
</table>

En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores( que tiene relación directa con la audiencia). Hay **3 equipos** en la **categoría A**, **11 equipos** de **categoría B** y **6 equipos** de **categoría C**.

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos).

|  <!-- -->       | Categoría A | Categoría B  | Categoría C   |
|-----------------|-------------|--------------|---------------|
| **Categoría A** | 2 Millones  | 1.3 Millones | 1 Millones    |
| **Categoría B** |  <!-- -->   | 0.9 Millones | 0.75 Millones |
| **Categoría C** |  <!-- -->   | <!-- -->     | 0.47 Millones |

Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla.

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes.

| <!-- --> | Viernes | Sábado  | Domingo | Lunes |
|----------|---------|---------|---------|-------|
| **12h**  | -       | 0.55    | 0.45    | -     |
| **16h**  | -       | 0.7     | 0.75    | -     |
| **18h**  | -       | 0.8     | 0.85    | -     |
| **20h**  |  0.4    | 1       | 1       | 0.4   |

Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

| Coincidencias | _%  |
|---------------|-----|
| 0             | 0%  |
| 1             | 25% |
| 2             | 45% |
| 3             | 60% |
| 4             | 70% |
| 5             | 75% |
| 6             | 78% |
| 7             | 80% |
| 8             | 80% |

Los cálculos asociados a una jornada de ejemplo se realizan según se muestra en la siguiente tabla:

| Partido|Categorías|Horario|Base(Mill.)|Ponderación|Base*Ponderación|Corrección Coincidencia|
|----------------------|-----|-----|-----|-----|-----|-----|
|Celta - Real Madrid   | B-A | V20 | 1,3 | 0,4 | 0,52| 0,52|
|Valencia - R. Sociedad| B-A | S12 | 1,3 | 0,55| 0,72| 0,72|
|Mallorca - Eibar      | C-C | S16 | 0.47| 0,7 | 0,33| 0,33|
|Athletic - Barcelona  | B-A | S18 | 1,3 | 0,8 | 1,04| 1,04|
|Leganés - Osasuna     | C-C | S20 | 0,47| 1   | 0,47| 0,47|
|Villareal - Granada   | B-C | D16 | 0,75| 0,75| 0,56| 0,42|
|Alavés - Levante      | B-B | D16 | 0,9 | 0,75| 0,68| 0,51|
|Espanyol - Sevilla    | B-B | D18 | 0,9 | 0,85| 0,77| 0,77|
|Bétis - Valladolid    | B-C | D20 | 0,75| 1   | 0,75| 0,75|
|Atlético - Getafe     | B-B | L20 | 0,9 | 0,4 | 0,36| 0,36|

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

- **¿Como represento el espacio de soluciones?**

El espacio de soluciones viene dado por el conjunto de partidos P y el conjunto de horarios H los cuales se agrupan como un diccionario:

`
{('Celta', 'Betis'): 'D18', ('Villarreal', 'Sevilla'): 'L20', ...},
`

donde las claves son los partidos que se juegan en la jornada y los valores el horario junto a la inicial del día. Por lo tanto, el número de combinaciones posibles es:

$$
Combinaciones = \binom{N}{10},
$$

$N$ hace referencia al número de partidos totales posibles.


- **¿Cual es la función objetivo?**

En este problema la función objetivo es maximizar la audiencia total. Por lo tanto la función objetivo es:
$$ f_{objetivo} = max \sum_{j \in H} R(k_j) \sum_{i\in P} x_{i,j} \cdot A_i \cdot C_j,$$

donde, $A_i$ es la audiencia base del partido según a la categoría que pertenece cada equipo. $C_j$ es el coeficiente de penalización dependiendo la franja horaria. $R(k_j)$ es la penalización de audiencia debido a coincidencias en el mismo horario.

- **¿Como implemento las restricciones?**

Estas son las restricciones que se deben imponer al modelo:

1. Cada partido debe jugarse en un único horario:
$$ \sum_{j\in H} x_{i,j} = 1, \forall i \in P.$$

2. Un mismo equipo solo puede jugar un partido por jornada:
$$\sum_{i\in P_e} \sum_{j\in H} x_{i,j} \leq 1, \forall e \in E,$$
donde E es el conjunto de equipos y $P_e$ el conjunto de partidos en los que juega el equipo $e$.

3. Se ha de jugar un partido el viernes y otro el lunes:

$$\sum_{i\in P} x_{i,V20} = 1, \sum_{i\in P} x_{i,L20} = 1.$$

4. Reducción de audiencia por coincidencia:

| Coincidencias ($k_j$) | _%  | $R(k_j)$  |
|-----------------------|-----|-----------|
| 0                     | 0%  | 0.0       |
| 1                     | 25% | 0.25      |
| 2                     | 45% | 0.45      |
| 3                     | 60% | 0.60      |
| 4                     | 70% | 0.70  |
| 5                     | 75% | 0.75  |
| 6                     | 78% | 0.78 |
| 7                     | 80% | 0.8  |
| 8                     | 80% | 0.8  |




#Análisis
- **¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**


Tenemos **20 equipos** en la liga y cada jornada cuenta con **10 partidos**. Cada equipo solo puede jugar un partido por jornada. Contamos con $m$ opciones de horario. Por lo tanto el espacio de soluciones es:
$$
\binom{N}{10} \times 10!
$$

Para $N \approx 100$ partidos posibles, el espacio de soluciones es inmanejable por fuerza bruta. La **complejidad** del problema es **exponencial** $O(m^n)$, clasificándose como **NP-difícil**.

#Diseño
- **¿Que técnica utilizo? ¿Por qué?**

Al tratarse de un problema de combinatoria la resolución por fuerza bruta se vuelve inviable. Por ello, podremos resolverlo mediante métodos voraces vistos en clase como divide y vencerás y vuelta atrás o con algoritmos genéticos.

Entre los diferentes métodos propuestos divide y vencerás se dificulta a la hora de dividir el problema principal en subpreblemas independientes. El método vuelta atrás al imponer tantas restricciones el coste computacional se eleva. Por ello, nos hemos decantado por **algoritmos genéticos** debido a que se aplican muchas condiciones y restricciones complejas. Este método encuentra buenas soluciones explorando múltiples soluciones simultáneamente utilizando mutaciones y cruce para mejorar los resultados. Además, tiene menor coste computacional que los métodos mencionados anteriormente∫.

In [19]:
import random
###### DEFINIMOS LAS VARIABLES DADAS ######
# Los diferentes posibles horarios
horarios = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']

# Definimos los equipos en base a la categoria a la que pertenecen
equipos = {
    'A': ['Real Madrid', 'R. Sociedad', 'Barcelona'],
    'B': ['Celta', 'Valencia', 'Athletic', 'Villarreal', 'Alavés', 'Levante', 'Espanyol', 'Sevilla', 'Betis', 'Atlético', 'Getafe'],
    'C': ['Mallorca', 'Eibar', 'Leganés', 'Osasuna', 'Granada', 'Valladolid']
}

# La audiencia base dependiendo las categorias de los equipos
audiencia_base = {
    ('A', 'A'): 2.0,
    ('A', 'B'): 1.3,
    ('A', 'C'): 1.0,
    ('B', 'B'): 0.9,
    ('B', 'C'): 0.75,
    ('C', 'C'): 0.47
}

# Factor penalizador dependiendo el horario de partido
penalizacion_horario = {
    'V20': 0.4,
    'S12': 0.55,
    'S16': 0.7,
    'S18': 0.8,
    'S20': 1,
    'D12': 0.45,
    'D16': 0.75,
    'D18': 0.85,
    'D20': 1,
    'L20': 0.4
}

# Factor penalizador debido a coincidencias de partidos simultáneos
penalizacion_coincidencia = [0, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80]


###### GENERACIÓN DE SOLUCIONES POSIBLES ######

partidos = []
for i in range(len(equipos['A'])):
    for j in range(i + 1, len(equipos['A'])):
        partidos.append((equipos['A'][i], equipos['A'][j]))
for equipo_A in equipos['A']:
    for equipo_B in equipos['B']:
        partidos.append((equipo_A, equipo_B))
for equipo_A in equipos['A']:
    for equipo_C in equipos['C']:
        partidos.append((equipo_A, equipo_C))
for i in range(len(equipos['B'])):
    for j in range(i + 1, len(equipos['B'])):
        partidos.append((equipos['B'][i], equipos['B'][j]))
for equipo_B in equipos['B']:
    for equipo_C in equipos['C']:
        partidos.append((equipo_B, equipo_C))
for i in range(len(equipos['C'])):
    for j in range(i + 1, len(equipos['C'])):
        partidos.append((equipos['C'][i], equipos['C'][j]))


###### TABLA DE RESULTADOS ######
def generar_tabla(asignacion):
    tabla = []
    horarios_usados = {h: 0 for h in horarios}

    for partido, horario in asignacion.items():
        cat1 = 'A' if partido[0] in equipos['A'] else ('B' if partido[0] in equipos['B'] else 'C')
        cat2 = 'A' if partido[1] in equipos['A'] else ('B' if partido[1] in equipos['B'] else 'C')
        audiencia_base_partido = audiencia_base[(cat1, cat2)]
        ponderacion = round(penalizacion_horario[horario],2)
        base_ponderada = round(audiencia_base_partido * ponderacion,2)
        horarios_usados[horario] += 1

        tabla.append([f"{partido[0]} - {partido[1]}", f"{cat1}-{cat2}", horario, audiencia_base_partido, ponderacion, base_ponderada])

    for fila in tabla:
        coincidencias = horarios_usados[fila[2]] - 1
        correccion = fila[5] * (1 - penalizacion_coincidencia[coincidencias]) if coincidencias > 0 else fila[5]
        fila.append(correccion)

    return tabla



###### ALGORITMO GENÉTICO ######

def algoritmo_genetico(poblacion_size=100, generaciones=500):
    poblacion = []
    for _ in range(poblacion_size):
        asignacion = {}
        partidos_disponibles = random.sample(partidos, len(partidos))
        equipos_usados = set()
        # cada jornada ha de tener 10 partidos
        # los equipos no se pueden repetir en una misma jornada
        for partido in partidos_disponibles:
            if len(asignacion) >= 10:
                break
            if partido[0] in equipos_usados or partido[1] in equipos_usados:
                continue
            asignacion[partido] = random.choice(horarios)
            equipos_usados.add(partido[0])
            equipos_usados.add(partido[1])

        if len(asignacion) == 10:
            poblacion.append(asignacion)

    # ordenamos la población según la audiencia total
    for _ in range(generaciones):
        poblacion = sorted(poblacion, key=lambda x: sum(row[-1] for row in generar_tabla(x)), reverse=True)
        nueva_poblacion = poblacion[:10]

        while len(nueva_poblacion) < poblacion_size:
            p1, p2 = random.sample(poblacion[:50], 2)
            hijo = {}
            equipos_usados = set()

            for partido in p1:
                if len(hijo) >= 10:
                    break
                if partido[0] in equipos_usados or partido[1] in equipos_usados:
                    continue
                hijo[partido] = random.choice(horarios)
                equipos_usados.add(partido[0])
                equipos_usados.add(partido[1])

            if len(hijo) == 10:
                nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

    mejor_solucion = max(poblacion, key=lambda x: sum(row[-1] for row in generar_tabla(x)))
    return generar_tabla(mejor_solucion)

tabla_resultado = algoritmo_genetico()
print("Partido\tCategorías\tHorario\tBase(Mill.)\tPonderación\tBase*Ponderación\tCorrección Coincidencia")
for fila in tabla_resultado:
    print("\t".join(map(str, fila)))

Partido	Categorías	Horario	Base(Mill.)	Ponderación	Base*Ponderación	Corrección Coincidencia
Real Madrid - Atlético	A-B	S18	1.3	0.8	1.04	1.04
Leganés - Granada	C-C	V20	0.47	0.4	0.19	0.19
Espanyol - Mallorca	B-C	S12	0.75	0.55	0.41	0.41
Celta - Valencia	B-B	D16	0.9	0.75	0.68	0.68
Athletic - Osasuna	B-C	D18	0.75	0.85	0.64	0.48
Getafe - Eibar	B-C	D18	0.75	0.85	0.64	0.48
Levante - Sevilla	B-B	S16	0.9	0.7	0.63	0.63
Betis - Valladolid	B-C	D12	0.75	0.45	0.34	0.34
Villarreal - Alavés	B-B	D20	0.9	1	0.9	0.9
R. Sociedad - Barcelona	A-A	S20	2.0	1	2.0	2.0
